In [1]:
import config as config
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    log_loss,
    roc_auc_score,
)
from sklearn.preprocessing import LabelBinarizer

This code is scratch work to train calibration of one single model outside of the main scripts, juts to check and compare training is doing what we think it's doing. A spot check. Also a few other techniques, like bin histogram, logistic, etc are also tried here but did not proceed. Mostly, this code is just scratch work. 

In [56]:
csv = f"/home/csutter/DRIVE-clean/CNN/data_preds/nestcv_5cat_twotrain_OT4_m2_T2V3__A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue.csv"

# read in data
dfread = pd.read_csv(csv)

print(dfread.columns)

t_all = dfread

# # prep column names
# t_all = calib.rename_cols_for_calibration_consistency(dfinput = dfread, classification_model = config.classif_model)

# add classifier col of 0s and 1s if model predicted that cat
t_all["classifier_TF"] = (
    t_all["img_cat"] == t_all["model_pred"]
    )
t_all["classifier_01"] = t_all["classifier_TF"].astype(int)

print(t_all.columns)

t_val = t_all[t_all["innerPhase"] == "innerVal"]
print(len(t_val))


X_unshaped = np.array(t_val["model_prob"])
X = X_unshaped.reshape(-1, 1)

y = np.array(t_val["classifier_01"])

X_eval_unshaped = np.array(t_all["model_prob"])
X_eval = X_eval_unshaped.reshape(-1, 1)

print(X_eval.shape)

iso_reg = IsotonicRegression(out_of_bounds="clip")
print("inside isot")
print(X[0:3])
print(y[0:3])

# WAY 1: isotonic
# iso_reg.fit(X, y)  # Fit isotonic regression on the probabilities
# # Use the calibrated probabilities to make predictions
# traindata_output_predProb = iso_reg.transform(X)  # Calibrate the probabilities
# evaldata_output_predProb = iso_reg.transform(X_eval)  # Calibrate the probabilities

# WAY 2: logistic
lr = LogisticRegression()
lr.fit(X, y)
evaldata_output_predProb = lr.predict_proba(X_eval)[:, 1]


print(evaldata_output_predProb)
print(type(evaldata_output_predProb))

# WAY 3: BINNING CALIBRATION
from sklearn.calibration import calibration_curve

prob_true, prob_pred = calibration_curve(y, X, n_bins=20, strategy='quantile')
print(prob_true)
print(prob_pred)

# apply these to 

# These come from sklearn.calibration.calibration_curve
# bin_means (prob_pred) and bin_actuals (prob_true) are each length `n_bins`
bin_means = np.array(prob_pred)   # predicted prob mean per bin
print("bin mean")
print(bin_means)
bin_actuals = np.array(prob_true) # actual accuracy per bin
print("bin_actuals")
print(bin_actuals)
# proportion method
bin_scale = bin_actuals/bin_means
print("bin_scale")
print(bin_scale)
# You have a long array of raw model probs (e.g., from model.predict)
raw_probs = np.array(X_eval)  # shape (n_samples,)

# Step 1: Digitize raw probs into bins
bin_edges = np.linspace(0.0, 1.0, len(bin_means) + 1)  # same as in calibration_curve
print("bin_edges")
print(bin_edges)

bin_indices = np.digitize(raw_probs, bin_edges[1:], right=True)  # bin index per sample
print("bin_actuals")
print(bin_actuals)

print("bin_indices")
print(len(bin_indices))
print(bin_indices)

# Step 2 - Method 1: Map each prob to the corresponding calibrated (empirical) value
calibrated_probs = bin_actuals[bin_indices]  # shape (n_samples,)
print("calibrated_probs ALL SAME PROB PER BIN")
print(calibrated_probs)

# Step 2 - Method 2: Scale everything. Assign proportion that the bin need to be adjusted downward, rather than fixed bin amounts
calibrated_probs = np.clip(raw_probs * bin_scale[bin_indices], 0, 1)
print("calibrated_probs DYNAMIC SCALE RATIO")
print(calibrated_probs)
print('check og')
print(raw_probs)




# print(np.unique(calibrated_probs))
# print(len(calibrated_probs))
# # Optional sanity check
# for i in range(3):
#     print(f"Original: {raw_probs[i]:.3f} --> Calibrated: {calibrated_probs[i]:.3f}")


Index(['Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
       'innerPhase', 'Unnamed: 0', 'outerPhase', 'img_name', 'img_orig',
       'site', 'img_index', 'img_cat', 'foldnum', 'ID', 'Latitude',
       'Longitude', 'date', 'yr', 'mo', 'day', 'time', 'yr_mo', 'timeofday',
       'timeofevent', 'innerPhaseOrig', 'prob_dry', 'prob_poor_viz',
       'prob_snow', 'prob_snow_severe', 'prob_wet', 'model_pred', 'tracker',
       'model_prob'],
      dtype='object')
Index(['Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
       'innerPhase', 'Unnamed: 0', 'outerPhase', 'img_name', 'img_orig',
       'site', 'img_index', 'img_cat', 'foldnum', 'ID', 'Latitude',
       'Longitude', 'date', 'yr', 'mo', 'day', 'time', 'yr_mo', 'timeofday',
       'timeofevent', 'innerPhaseOrig', 'prob_dry', 'prob_poor_viz',
       'prob_snow', 'prob_snow_severe', 'prob_wet', 'model_pred', 'tracker',
       'model_prob', 'classifier_TF', 'classifier_01'],
      dtype='object')
3773
(21

In [45]:
len(bin_indices)

21489

In [46]:
len(bin_actuals)

20

In [47]:
len(calibrated_probs)

21489

In [34]:
t_all["log_calib"] = evaldata_output_predProb


t_all.to_csv("testcalib1.csv")


In [38]:
t_all["hist_calib"] = prob_true


t_all.to_csv("testcalib_hist.csv")

ValueError: Length of values (10) does not match length of index (21489)

In [57]:
t_all["calib_bin_scale"] = calibrated_probs


t_all.to_csv("testcalib_binscale.csv")

In [35]:
t_all.head(5)

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,innerPhase,Unnamed: 0,outerPhase,img_name,img_orig,site,...,prob_poor_viz,prob_snow,prob_snow_severe,prob_wet,model_pred,tracker,model_prob,classifier_TF,classifier_01,log_calib
0,0,0,0,0,NAOuterTest,0,outerTest,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,...,0.000238,0.011237,0.000569,0.947660,wet,OT4_m2_T2V3,0.947660,True,1,0.883929
1,1,1,1,1,NAOuterTest,1,outerTest,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,...,0.000466,0.035654,0.000598,0.862268,wet,OT4_m2_T2V3,0.862268,True,1,0.843812
2,2,2,2,2,NAOuterTest,2,outerTest,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,...,0.000291,0.024719,0.000505,0.923279,wet,OT4_m2_T2V3,0.923279,True,1,0.873488
3,3,3,3,3,NAOuterTest,3,outerTest,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,...,0.000109,0.031323,0.000204,0.915742,wet,OT4_m2_T2V3,0.915742,True,1,0.870101
4,4,4,4,4,NAOuterTest,4,outerTest,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_5630,...,0.000117,0.023900,0.000181,0.915846,wet,OT4_m2_T2V3,0.915846,True,1,0.870148


In [13]:
# check that ^ manual one against the one that was ran in the broader code

df_ran = pd.read_csv("/home/csutter/DRIVE-clean/calibration/calib_CNN_data/nestcv_5cat_twotrain_OT4_m2_T2V3__A_resnet_TRLETrue_ASTFalse_L20_001_DR0_4_E75_AugTrue.csv")

df_ran.head(4)

predarray = np.array(df_ran["calib_prob"])
print(type(predarray))

<class 'numpy.ndarray'>


In [30]:
same = predarray==evaldata_output_predProb
print(len(same))
print(sum(same))

df_ran["manual"] = evaldata_output_predProb
df_ran["diff_from_manual"] = same

df_ran.head(5)

df_diffs = df_ran[df_ran["diff_from_manual"]==False].reset_index()

print(len(df_diffs))

# display(df_diffs)

d2 = df_diffs["manual"]==df_diffs["calib_prob"]
df_diffs["diffs2"] = d2

# display(df_diffs)

diffsubt = df_diffs["manual"]-df_diffs["calib_prob"]
df_diffs["diffsubt"] = diffsubt

display(df_diffs)

print(sum(df_diffs["diffsubt"]))
print(np.max(df_diffs["diffsubt"]))

21489
21243
246


,index,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,innerPhase,Unnamed: 0,outerPhase,img_name,...,calib_prob_dry,calib_prob_poor_viz,calib_prob_snow,calib_prob_snow_severe,calib_prob_wet,calib_pred,manual,diff_from_manual,diffs2,diffsubt
0,50,50,50,50,50,50,NAOuterTest,50,outerTest,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,...,0.098872,0.164099,0.260874,0.047584,0.428571,wet,0.428571,False,False,5.551115e-17
1,71,71,71,71,71,71,NAOuterTest,71,outerTest,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,...,0.032157,0.428571,0.161998,0.016040,0.361233,poor_viz,0.428571,False,False,5.551115e-17
2,144,144,144,144,144,144,NAOuterTest,144,outerTest,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,...,0.270394,0.001314,0.428571,0.000344,0.299378,snow,0.428571,False,False,5.551115e-17
3,254,254,254,254,254,254,NAOuterTest,254,outerTest,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,...,0.132528,0.122751,0.254904,0.061245,0.428571,wet,0.428571,False,False,5.551115e-17
4,306,306,306,306,306,306,NAOuterTest,306,outerTest,I_290_between_Exit_7_(Main_Street)_and_Exit_6_...,...,0.226184,0.037884,0.428571,0.012081,0.295279,snow,0.428571,False,False,5.551115e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,21069,21069,21069,21069,21069,21069,innerTest,21069,inner5CV,I_90_at_Interchange_48_(Batavia)__Eastbound__S...,...,0.181306,0.428571,0.046603,0.046019,0.297501,poor_viz,0.428571,False,False,5.551115e-17
242,21107,21107,21107,21107,21107,21107,innerTest,21107,inner5CV,I_90_at_Interchange_48_(Batavia)__Eastbound__S...,...,0.159216,0.133504,0.194263,0.160076,0.352941,wet,0.352941,False,False,5.551115e-17
243,21291,21291,21291,21291,21291,21291,innerTest,21291,inner5CV,I_495_at_Nicolls_Rd__Unknown__Skyline_1882_202...,...,0.089300,0.352941,0.142599,0.141239,0.273921,poor_viz,0.352941,False,False,5.551115e-17
244,21374,21374,21374,21374,21374,21374,innerTrain,21374,inner5CV,I_495_at_Sagtikos_Pkwy__Westbound__Skyline_569...,...,0.352941,0.024968,0.247293,0.090855,0.283943,dry,0.352941,False,False,5.551115e-17


1.3027773304585821e-14
1.1102230246251565e-16
